# 1️⃣ Column Renaming & Restructuring



In [ ]:
import pandas as pd
df = pd.read_csv('/content/customer_transactions.csv')

# convert all column names to snake case
df.columns = df.columns.str.lower().str.replace(' ($)', '').str.replace(' (%)', '').str.replace(' ', '_')
df.columns

Index(['transaction_id', 'customer_id', 'product_category', 'product_name',
       'quantity', 'unit_price', 'total_price', 'discount_applied',
       'purchase_date', 'payment_method'],
      dtype='object')

In [ ]:
# converting the date column to datetime format
df['purchase_date'] = pd.to_datetime(
    df['purchase_date'], format='mixed', dayfirst= False
    )

# converted payment methods to snakecase
df['payment_method'] = df['payment_method'].str.lower().str.replace(' ', '_')
df['payment_method'].unique()

array(['bank_transfer', 'credit_card', 'paypal'], dtype=object)

# 2️⃣ Creating & Modifying Columns

In [ ]:
# Fill missing values in Total_Price by multiplying Quantity × Unit_Price.
new_total_price = df['quantity'] * df['unit_price']
df['total_price'].fillna(new_total_price, inplace=True)

# Fill missing values in Discount_Applied with 0%.
df['discount_applied'].fillna('0%', inplace=True)
df['discount_applied'] = df['discount_applied'].str.replace('%', '').astype(float)/100

# Create a new column Final_Price that applies the discount
df['final_price'] = df['total_price'] * (1-df['discount_applied'])

df['purchase_year'] = df['purchase_date'].dt.year
df['purchase_month'] = df['purchase_date'].dt.month_name()



<ipython-input-78-45133a360f02>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['total_price'].fillna(new_total_price, inplace=True)
<ipython-input-78-45133a360f02>:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', t

# 3️⃣ Using `.apply()` & `lambda` Functions

In [ ]:
# creating a purchase type column
df['purchase_type'] = df['quantity'].apply(
    lambda x: 'bulk' if x > 5 else 'single item'
    )

# categorizing unit price
df['price_category'] = df['unit_price'].apply(
    lambda price: 'Low' if price < 20 else ('Medium' if price <= 100 else 'High')
    )

# standardizing product category
df['product_category'] = df['product_category'].apply(lambda x: x.capitalize())

# flagging high value transactions
df['transaction_flag'] = df['final_price'].apply(lambda x: 'High Value' if x > 100 else 'Low Value')

In [ ]:
df.head()

,transaction_id,customer_id,product_category,product_name,quantity,unit_price,total_price,discount_applied,purchase_date,payment_method,final_price,purchase_year,purchase_month,purchase_type,price_category
0,10001,2103,Toys,Action Figure,9,58.76,528.84,0.10,2023-07-16,bank_transfer,475.956,2023,July,bulk,Medium
1,10002,2436,Books,Non-Fiction,10,239.03,2390.30,0.05,2023-06-18,credit_card,2270.785,2023,June,bulk,High
2,10003,2861,Toys,Lego,5,340.18,1700.90,0.10,2024-02-07,bank_transfer,1530.810,2024,February,single item,High
3,10004,2271,Beauty,Skincare Set,7,437.88,3065.16,0.00,2024-11-12,bank_transfer,3065.160,2024,November,bulk,High
4,10005,2107,Beauty,Foundation,9,375.86,3382.74,0.00,2023-09-18,paypal,3382.740,2023,September,bulk,High


# 4️⃣ Reshaping Data (Pivot, Melt, Stack, Unstack)

In [86]:
# using the melt method
var_columns = ['unit_price', 'final_price', 'total_price']
id_columns = [col for col in df.columns if col not in var_columns]
melted_df = df.melt(
    id_vars=id_columns, value_vars=var_columns,
    var_name='price_type', value_name='price_value'
    )
melted_df.head()

,transaction_id,customer_id,product_category,product_name,quantity,discount_applied,purchase_date,payment_method,purchase_year,purchase_month,purchase_type,price_category,transaction_flag,price_type,price_value
0,10001,2103,Toys,Action Figure,9,0.10,2023-07-16,bank_transfer,2023,July,bulk,Medium,High Value,unit_price,58.76
1,10002,2436,Books,Non-Fiction,10,0.05,2023-06-18,credit_card,2023,June,bulk,High,High Value,unit_price,239.03
2,10003,2861,Toys,Lego,5,0.10,2024-02-07,bank_transfer,2024,February,single item,High,High Value,unit_price,340.18
3,10004,2271,Beauty,Skincare Set,7,0.00,2024-11-12,bank_transfer,2024,November,bulk,High,High Value,unit_price,437.88
4,10005,2107,Beauty,Foundation,9,0.00,2023-09-18,paypal,2023,September,bulk,High,High Value,unit_price,375.86


In [95]:
# Use .pivot() to restructure data by showing Total_Sales per Customer_ID.
pivoted_data = df.pivot_table(
    index='customer_id',
    values='total_price', aggfunc = 'sum')
pivoted_data.head()

# stacking the data
stacked_data = df.stack()
stacked_data.head()

,total_price
customer_id,
2001,11748.66
2002,4501.73
2003,4395.55
2004,2733.06
2005,6685.93


In [96]:
# Average Final Price per Product_Category table
avg_price_by_category = df.pivot_table(index = 'product_category',
                                       values = 'final_price',
                                       aggfunc = 'mean')
avg_price_by_category.head()

# Total Discount Applied per Payment_Method summary method
total_discount_by_method = df.pivot_table(index = 'payment_method',
                                          values = 'discount_applied',
                                          aggfunc = 'sum')
total_discount_by_method.head()

,discount_applied
payment_method,
bank_transfer,130.45
credit_card,133.20
paypal,114.75


# 5️⃣ Handling Duplicate & Missing Data

In [98]:
df.duplicated().sum()

np.int64(0)

In [99]:
df[df.duplicated()]

,transaction_id,customer_id,product_category,product_name,quantity,unit_price,total_price,discount_applied,purchase_date,payment_method,final_price,purchase_year,purchase_month,purchase_type,price_category,transaction_flag


In [102]:
from google.colab import files
df.to_csv('transformed_customer_data.csv', index=False)

# downloading cleaned data
files.download('transformed_customer_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>